In [244]:
import pandas as pd
import numpy as np
import warnings
warnings.simplefilter('ignore')
pd.set_option('display.max_columns', None)
import pgeocode
nomi = pgeocode.Nominatim('IN')
import matplotlib.pyplot as plt

In [245]:
invoice=pd.read_csv("C:\\Users\\Neel\\Desktop\\GreyAtom Capstone Project\\Final_invoice.csv")
invoice.head()

,Unnamed: 0,Amt Rcvd From Custom,Amt Rcvd From Ins Co,Area / Locality,CGST(14%),CGST(2.5%),CGST(6%),CGST(9%),CITY,Cash /Cashless Type,Claim No.,Cust Type,Customer No.,District,Expiry Date,Gate Pass Date,Gate Pass Time,IGST(12%),IGST(18%),IGST(28%),IGST(5%),Insurance Company,Invoice Date,Invoice No,Invoice Time,Job Card No,JobCard Date,JobCard Time,KMs Reading,Labour Total,Make,Misc Total,Model,ODN No.,OSL Total,Order Type,Outstanding Amt,Parts Total,Pin code,Plant,Plant Name1,Policy no.,Print Status,Recovrbl Exp,Regn No,SGST/UGST(14%),SGST/UGST(2.5%),SGST/UGST(6%),SGST/UGST(9%),Service Advisor Name,TDS amount,Technician Name,Total Amt Wtd Tax.,Total CGST,Total GST,Total IGST,Total SGST/UGST,Total Value,User ID
0,0,0.0,0.0,MAJIWADA,0.0,0.0,0.0,0.0,Thane,NaN,NaN,Retail,67849,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-02,7005200002,11:30:36,168303,2014-12-13,14:29:43,49317,1203.14,GENERAL MOTORS,0.00,SPARK,7.005200e+09,500.06,Paid Service,0.0,2348.75,400601,BC01,THANE,NaN,NO,0.0,KA19MA1291,0.0,0.0,0.0,0.0,NaN,0.0,RUPESH,4051.95,0.0,0.0,0.0,0.0,4051.95,BC01FS1
1,1,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200003,10:07:32,173997,2015-01-02,14:12:18,78584,804.26,TATA MOTORS,197.03,INDICA,7.005200e+09,0.00,SMC Value Package,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,1001.29,0.0,0.0,0.0,0.0,1001.29,BC01SA2
2,2,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE[W],NaN,NaN,Retail,81055,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200004,11:12:57,173889,2015-01-02,11:40:44,33985,180.19,MARUTI SUZUKI,0.00,ZEN,7.005200e+09,0.00,Running Repairs,0.0,52.95,400607,BC01,THANE,NaN,NO,0.0,AP09AX0582,0.0,0.0,0.0,0.0,NaN,0.0,IMRAN,233.14,0.0,0.0,0.0,0.0,233.14,BC01SA2
3,3,0.0,0.0,THNAE,0.0,0.0,0.0,0.0,THNAE,NaN,NaN,Retail,84419,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200005,11:40:44,174158,2015-01-03,10:12:32,78584,0.00,TATA MOTORS,0.00,INDICA,7.005200e+09,0.00,SMC Redemption,0.0,0.00,400603,BC01,THANE,NaN,NO,0.0,MH43R3046,0.0,0.0,0.0,0.0,NaN,0.0,PRASHANT,0.00,0.0,0.0,0.0,0.0,0.00,BC01SA2
4,4,0.0,0.0,THANE,0.0,0.0,0.0,0.0,THANE,NaN,NaN,Retail,18980,Maharashtra,NaN,NaN,00:00:00,0.0,0.0,0.0,0.0,NaN,2015-01-03,7005200006,12:07:17,173860,2015-01-02,10:45:42,50057,1202.97,HYUNDAI,100.00,SANTRO XING,7.005200e+09,499.99,Paid Service,0.0,944.16,400603,BC01,THANE,NaN,NO,0.0,MH04CD9768,0.0,0.0,0.0,0.0,NaN,0.0,DAYANAND,2747.12,0.0,0.0,0.0,0.0,2747.12,BC01SA2


In [246]:
invoice["Clean_CITY"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).county_name
invoice["Clean_District"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).state_name
invoice["Clean_Location_Area"]=nomi.query_postal_code(np.array(invoice["Pin code"].astype(str))).place_name

In [247]:
invoice.drop(["Area / Locality","CITY","District"],1,inplace=True)

In [248]:
invoice["Invoice_DateTime"]=pd.to_datetime(invoice["Invoice Date"]+" "+invoice["Invoice Time"])
invoice["JobCard_DateTime"]=pd.to_datetime(invoice["JobCard Date"]+" "+invoice["JobCard Time"])

In [249]:
invoice.drop(["Invoice Date","Invoice Time","JobCard Date","JobCard Time"],1,inplace=True)

In [250]:
invoice["Service_Time"]=invoice["Invoice_DateTime"]-invoice["JobCard_DateTime"]

In [251]:
invoice["Service_Time_Hours"]=invoice["Service_Time"]/np.timedelta64(1, 'h')

In [252]:
invoice["Year"]=invoice["Invoice_DateTime"].dt.year
invoice["Day"]=invoice["Invoice_DateTime"].dt.day
invoice["Month"]=invoice["Invoice_DateTime"].dt.month

In [253]:
invoice_13=invoice[(invoice["Year"]==2013)&(invoice["Order Type"]=='Running Repairs')]

plant_tech_total=invoice_13.groupby(["Plant"]).agg({"Technician Name":"count","Job Card No":'count',"Invoice No":'count',"Service_Time_Hours":'sum',"Total Value":'sum'})
plant_tech_total["Amt/Hour"]=plant_tech_total["Total Value"]/plant_tech_total["Service_Time_Hours"]
plant_tech_total["Productivity"]=plant_tech_total["Amt/Hour"]/plant_tech_total["Technician Name"]

plant_prod_2013=plant_tech_total.reset_index().rename(columns={"Productivity": "Productivity (in 2013)"})
plant_prod_2013.head()

,Plant,Technician Name,Job Card No,Invoice No,Service_Time_Hours,Total Value,Amt/Hour,Productivity (in 2013)
0,BC01,1254,1824,1824,73423.079167,5429736.28,73.951356,0.058972
1,BC02,912,1171,1171,85211.368333,3266452.83,38.333533,0.042032
2,BC03,2628,3303,3303,80828.433611,8883730.83,109.908487,0.041822
3,BC04,710,1044,1044,38032.378889,3998253.16,105.127612,0.148067
4,BC05,1354,1993,1993,90923.263056,6344601.95,69.779743,0.051536


In [254]:
invoice_14=invoice[(invoice["Year"]==2014)&(invoice["Order Type"]=='Running Repairs')]

plant_tech_total=invoice_14.groupby(["Plant"]).agg({"Technician Name":"count","Job Card No":'count',"Invoice No":'count',"Service_Time_Hours":'sum',"Total Value":'sum'})
plant_tech_total["Amt/Hour"]=plant_tech_total["Total Value"]/plant_tech_total["Service_Time_Hours"]
plant_tech_total["Productivity"]=plant_tech_total["Amt/Hour"]/plant_tech_total["Technician Name"]

plant_prod_2014=plant_tech_total.reset_index().rename(columns={"Productivity": "Productivity (in 2014)"})
plant_prod_2014.head()

,Plant,Technician Name,Job Card No,Invoice No,Service_Time_Hours,Total Value,Amt/Hour,Productivity (in 2014)
0,BC01,1390,1413,1413,68902.650833,4642809.29,67.382158,0.048476
1,BC02,1097,1111,1111,58195.806111,3220834.87,55.344793,0.050451
2,BC03,4282,4303,4303,143791.818333,12372636.70,86.045485,0.020095
3,BC04,344,344,344,11012.165833,1739014.79,157.917599,0.459063
4,BC05,2432,2438,2438,114891.672222,12890021.37,112.192826,0.046132


In [256]:
invoice_15=invoice[(invoice["Year"]==2015)&(invoice["Order Type"]=='Running Repairs')]

plant_tech_total=invoice_15.groupby(["Plant"]).agg({"Technician Name":"count","Job Card No":'count',"Invoice No":'count',"Service_Time_Hours":'sum',"Total Value":'sum'})
plant_tech_total["Amt/Hour"]=plant_tech_total["Total Value"]/plant_tech_total["Service_Time_Hours"]
plant_tech_total["Productivity"]=plant_tech_total["Amt/Hour"]/plant_tech_total["Technician Name"]

plant_prod_2015=plant_tech_total.reset_index().rename(columns={"Productivity": "Productivity (in 2015)"})
plant_prod_2015.head()

,Plant,Technician Name,Job Card No,Invoice No,Service_Time_Hours,Total Value,Amt/Hour,Productivity (in 2015)
0,BC01,1261,1265,1265,59808.810278,3356174.62,56.115054,0.044500
1,BC02,177,177,177,57713.250278,538414.10,9.329125,0.052707
2,BC03,2553,2569,2569,112516.098889,5093748.04,45.271282,0.017733
3,BC05,1066,1072,1072,75006.013611,4876460.74,65.014264,0.060989
4,BC06,1338,1359,1359,75961.328056,5623477.12,74.030790,0.055329


In [257]:
productivities=pd.merge(plant_prod_2013[["Plant","Productivity (in 2013)"]],plant_prod_2014[["Plant","Productivity (in 2014)"]],on="Plant").merge(plant_prod_2015[["Plant","Productivity (in 2015)"]],on="Plant")

In [260]:
productivities

,Plant,Productivity (in 2013),Productivity (in 2014),Productivity (in 2015)
0,BC01,0.058972,0.048476,0.044500
1,BC02,0.042032,0.050451,0.052707
2,BC03,0.041822,0.020095,0.017733
3,BC05,0.051536,0.046132,0.060989
4,BC06,0.074857,0.047409,0.055329
5,BC07,0.061915,0.039945,0.039466
6,BC08,0.075407,0.042338,0.051045
7,BC12,0.062201,0.064378,0.100813
8,BC13,0.072496,0.035552,0.056383
9,BC15,0.056286,0.035543,0.057437
